<a href="https://colab.research.google.com/github/ysnarslan/GAN/blob/main/AdvancedGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import Libraries

import torch, torchvision, os, PIL, pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
def show(tensor, num=25, wandbactive=0, name=''):
    data = tensor.deatch()
    grid = make_grid(data[:num], nrow=5).permute(1, 2, 0)

    if(wandbactive and wandb):
        wandbactive.log({name: wandbactive.Image(grid.numpy().clip(0, 1))})

    plt.imshow(grid).clip(0, 1)
    plt.show()

In [5]:
# Hyperparameters & General parameters
n_epoch = 100
batch_size = 128
lr = 1e-4
z_dim = 200
device = 'cuda' # GPU

cur_step = 0
crit_cycles = 5
gen_losses = []
crit_losses = []
show_step = 35
save_step = 35
wandb = 1 # Track in wandb, optinal (Track: 1, Not track: 0)

In [1]:
## Optional
!pip install wandb -qqq
import wandb
wandb.login(key='') # Enter your API key.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
%%capture
experiment_name = wandb.util.generate_id()

myrun = wandb.init(
    project = 'wgan',
    group = experiment_name,
    config = {
        'optimizer': 'adam',
        'model': 'wgan gp',
        'batch_size': 128,
        'epoch': 1000,
    }
)

config = wandb.config

In [5]:
print(experiment_name)

bqsqnsed


In [6]:
# Generator Model
class Generator(nn.Module):
    def __init__(self, z_dim=64, d_dim=16):
        super(Generator, self).__init__()
        self.z_dim = z_dim

        self.gen = nn.Sequential(
            # inp = 1x1x200 (x_dim) -> out = 4x4x512 (d_dim * 32)
            nn.ConvTranspose2d(in_channels = z_dim,
                               out_channels = d_dim*32,
                               kernel_size = 4,
                               stride = 1,
                               padding = 0),
            nn.BatchNorm2d(d_dim * 32),
            nn.ReLU(inplace=True),

            # 4x4x512 -> 8x8x256
            nn.ConvTranspose2d(in_channels = d_dim*32,
                               out_channels = d_dim*16,
                               kernel_size = 4,
                               stride = 2,
                               padding = 1),
            nn.BatchNorm2d(d_dim * 16),
            nn.ReLU(True),

            # in = 8x8x256 -> out = 16x16x128
            nn.ConvTranspose2d(in_channels = d_dim * 16,
                               out_channels = d_dim * 8,
                               kernel_size = 4,
                               stride = 2,
                               padding = 1),
            nn.BatchNorm2d(d_dim * 8),
            nn.ReLU(True),

            # in = 16x16x128 -> out = 32x32x64
            nn.ConvTranspose2d(in_channels = d_dim * 8,
                               out_channels = d_dim * 4,
                               kernel_size = 4,
                               stride = 2,
                               padding = 1),
            nn.BatchNorm2d(d_dim * 4),
            nn.ReLU(True),

            # in = 32x32x64 -> out = 64x64x32
            nn.ConvTranspose2d(in_channels = d_dim * 4,
                               out_channels = d_dim * 2,
                               kernel_size = 4,
                               stride = 2,
                               padding = 1),
            nn.BatchNorm2d(d_dim * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels = d_dim * 2,
                               out_channels = 3,
                               kernel_size = 4,
                               stride = 2,
                               padding = 1),
            # Produce result in the range from -1 to 1
            nn.Tanh()
        )

    def forward(self, noise):
        x = noise.view(len(noise), self.z_dim, 1, 1) # 128x200x1x1
        return self.gen(x)

def gen_noise(num, z_dim, device='cuda'):
    return torch.randn(num, z_dim, device=device) # 128x200

In [10]:
gen = Generator(z_dim = 200)

gen

Generator(
  (gen): Sequential(
    (0): ConvTranspose2d(200, 512, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats